In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split
from joblib import dump , load

# import os
# import json
# import requests
# from azureml.core import Workspace
# from azureml.core.model import Model
# from azureml.core.environment import Environment
# from azureml.core.conda_dependencies import CondaDependencies
# from azureml.core.model import InferenceConfig
# from azureml.core.webservice import AciWebservice, Webservice

In [267]:
# from azureml.core import Environment
# from azureml.core.conda_dependencies import CondaDependencies

# env = Environment(name="recommender-env")
# deps = CondaDependencies.create(pip_packages=["pandas", "numpy", "scikit-learn", "surprise", "nltk", "azureml-defaults"])
# env.python.conda_file = "myenv.yml"  # Alternatively create a YAML file with dependencies

In [268]:
# from azureml.core import Model

# model = Model.register(workspace=workspace,
#                         model_name='recommender_model',
#                         model_path='model/recommender_model.joblib')

In [269]:
# from azureml.core.model import InferenceConfig

# inference_config = InferenceConfig(entry_script="score.py", environment=env)


In [270]:
# from azureml.core.webservice import AciWebservice, Webservice

# deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
# service = Model.deploy(workspace=workspace,
#                         name='recommender-service',
#                         models=[model],
#                         deployment_config=deployment_config,
#                         inference_config=inference_config)
# service.wait_for_deployment(show_output=True)


In [271]:
# import requests

# url = service.scoring_uri
# input_data = json.dumps({"data": {"userId": 1, "title": "Spider-Man"}})
# headers = {"Content-Type": "application/json"}
# response = requests.post(url, input_data, headers=headers)
# print(response.json())

In [272]:
# Loading the movies metadata Dataset

df = pd.read_csv('dataset/movies_metadata.csv', low_memory=False)
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [273]:
# Checking out what kind of Genres we have and filling out the empty genres with empty list ( It was Json previouesly )
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [274]:
# Getting the mean of the vote_averge and converting the vote_counts to int so it is easier to deal with 

vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [275]:
# Getting the 0.85 quantile of the vote counts

m = vote_counts.quantile(0.85)
m

82.0

In [276]:
# Here we extract the realse Year of the movie from the entire DateTime , and if not there makes it NaN

df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df['year'] 

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45461     NaT
45462    2011
45463    2003
45464    1917
45465    2017
Name: year, Length: 45466, dtype: object

In [277]:
# We select the movies which has the vote count of atleast the 0.95 quantile ( which is 434 ) and if it is below that we consider it not qualified
qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

# We convert the type of the vote_count and vote_averge to int ( since we did that above so we will need to do that again to the new part of the dataset we made )
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(6832, 6)

In [278]:
# Gives weights to each movie depending on thier vote_count and thier vote_averge using some Formula 

def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']                
    return (v/(v+m) * R) + (m/(m+v) * C)  



In [279]:
# Applying the weight_rating function to the qualifted Movies

qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [280]:
# Printing the qualifed movies to check out the wr for them
qualified

,title,year,vote_count,vote_average,popularity,genres,wr
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]",6.973819
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]",5.975183
2,Grumpier Old Men,1995,92,6,11.7129,"[Romance, Comedy]",5.644147
4,Father of the Bride Part II,1995,173,5,8.387519,[Comedy],5.078751
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]",6.926871
...,...,...,...,...,...,...,...
45268,"Good Guys Go to Heaven, Bad Guys Go to Pattaya",2016,153,5,5.613875,[Comedy],5.085453
45339,With Open Arms,2017,94,5,7.250495,[Comedy],5.114100
45343,The Visitors: Bastille Day,2016,167,4,7.29492,[Comedy],4.409966
45437,In a Heartbeat,2017,146,8,20.82178,"[Family, Animation, Romance, Comedy]",7.009129


In [281]:
# Sorting the movies upon thier wr 

qualified = qualified.sort_values('wr', ascending=False)

Top 10 Movies

In [282]:
# The top 10 movies with highest wr

qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,"[Comedy, Drama, Romance]",8.585574
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.984042
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.981708
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.979952
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.976853
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.974825
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.974187
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.973232
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.972807
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.972546


In [283]:
# Instead of having each movie having one row that has all genres we made it into mutliple rows so it is more friendly to deal with and then he droped the genres ( the one that has all o them in one row )

s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name= 'genre'
gen_df = df.drop('genres', axis=1).join(s)

In [284]:
# Building a simple Recommender which simply suggests the best movies of each genre you chose

def build_chart(genre, percentile=0.85):
    df = gen_df[gen_df['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(weighted_rating , axis = 1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

# Top 10 movies of the genre you choose

In [285]:
build_chart('Romance')

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.585574
351,Forrest Gump,1994,8147,8,48.307194,7.972546
876,Vertigo,1958,1162,8,18.20822,7.818394
40251,Your Name.,2016,1030,8,34.461252,7.796836
883,Some Like It Hot,1959,835,8,11.845107,7.753633
...,...,...,...,...,...,...
7582,How to Steal a Million,1966,133,7,12.389626,6.330612
6912,Beauty and the Beast,1946,133,7,4.798852,6.330612
24888,Remember Sunday,2013,131,7,11.190606,6.324326
2801,Body Heat,1981,128,7,5.991482,6.314674


In [286]:
links = pd.read_csv('dataset/links_small.csv')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [287]:
df = df.drop([19730, 29503, 35587])

In [288]:
credits = pd.read_csv('dataset/credits.csv')
keywords = pd.read_csv('dataset/keywords.csv')

In [289]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

In [290]:
df['id'] = df['id'].astype('int')

In [291]:
df.shape

(45463, 25)

In [292]:
df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')

In [293]:
sdf = df[df['id'].isin(links)]
sdf.shape

(9219, 28)

In [294]:
sdf.loc[:, 'cast'] = sdf['cast'].apply(literal_eval)
sdf.loc[:, 'crew'] = sdf['crew'].apply(literal_eval)
sdf.loc[:, 'keywords'] = sdf['keywords'].apply(literal_eval)

sdf.loc[:,'cast_size'] = sdf['cast'].apply(lambda x: len(x))
sdf.loc[:,'crew_size'] = sdf['crew'].apply(lambda x: len(x))

C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\297000521.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,'cast_size'] = sdf['cast'].apply(lambda x: len(x))
C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\297000521.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,'crew_size'] = sdf['crew'].apply(lambda x: len(x))


In [295]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [296]:
sdf.loc[:,'director'] = sdf['crew'].apply(get_director)

C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\4266608746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf.loc[:,'director'] = sdf['crew'].apply(get_director)


In [297]:
sdf.loc[:,'cast'] = sdf['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
sdf.loc[:,'cast'] = sdf['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [298]:
sdf.loc[:,'keywords'] = sdf['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [299]:
sdf.loc[:,'cast'] = sdf['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [300]:
sdf.loc[:,'director'] = sdf['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
sdf.loc[:,'director'] = sdf['director'].apply(lambda x: [x,x, x])

In [301]:
s = sdf.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [302]:
s = s.value_counts()
s[:5]

keyword
independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: count, dtype: int64

In [303]:
s = s[s > 1]

In [304]:
stemmer = SnowballStemmer('english')

In [305]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [306]:
sdf.loc[:,'keywords'] = sdf['keywords'].apply(filter_keywords)
sdf.loc[:,'keywords'] = sdf['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
sdf.loc[:,'keywords'] = sdf['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [307]:
sdf = sdf.copy()
sdf['soup'] = sdf['keywords'] + sdf['cast'] + sdf['director'] + sdf['genres']
sdf['soup'] = sdf['soup'].apply(lambda x: ' '.join(x))

In [308]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
count_matrix = count.fit_transform(sdf['soup']) 

In [309]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [310]:
smd = sdf.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [311]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [312]:
get_recommendations('The Dark Knight').head(10)

C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\3440557458.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_count'] = qualified['vote_count'].astype('int')
C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\3440557458.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['vote_average'] = qualified['vote_average'].astype('int')
C:\Users\medoo\AppData\Local\Temp\ipykernel_26008\3440557458.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,title,vote_count,vote_average,year,wr
7608,Inception,14075,8,2010,7.984042
8573,Interstellar,11187,8,2014,7.979952
6587,The Prestige,4510,8,2006,7.950802
3373,Memento,4168,8,2000,7.946843
7991,The Dark Knight Rises,9263,7,2012,6.984599
6186,Batman Begins,7511,7,2005,6.981046
1122,Batman Returns,1706,6,1992,5.965370
132,Batman Forever,1529,5,1995,5.012465
9004,Batman v Superman: Dawn of Justice,7189,5,2016,5.002762
1252,Batman & Robin,1447,4,1997,4.066764


In [313]:
ratings = pd.read_csv('dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [314]:
ratings_gn = pd.read_csv('dataset/Genrated ratings Data.csv')
ratings_gn.head()

,userId,movieId,timestamp,rating
0,510,133730,849588178,2.097580
1,510,148672,846457615,3.033221
2,513,146590,851162656,2.529508
3,510,132772,846242746,2.091977
4,514,146968,833662076,0.333745


In [315]:
ratings_gn.loc[ratings_gn['rating'] < 1, 'rating'] = 1
ratings_gn.loc[ratings_gn['rating'] > 5, 'rating'] = 5

In [316]:
ratings_gn['rating'].value_counts().sort_values(ascending=False)

rating
4.000000    22859
1.000000    16534
5.000000    16349
3.000000    16139
3.500000     8367
            ...  
2.963843        1
4.429167        1
4.944217        1
4.939470        1
2.448165        1
Name: count, Length: 85825, dtype: int64

In [317]:
ratings_all = pd.concat([ratings, ratings_gn])

In [318]:
ratings_all

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
182987,624,142536,2.0,1473962172
182988,624,140725,3.0,1462187665
182989,624,144714,1.5,1474972061
182990,624,143257,2.5,1461356507


In [319]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(ratings_all[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)

In [320]:
predictions = model.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9777


0.9776694841359036

In [321]:
# Saving the model Weights 

model_path = 'model/recommender_model.joblib'
dump(model, model_path)

['model/recommender_model.joblib']

In [322]:
# Load the model
model_path = 'model/recommender_model.joblib'
model = load(model_path)

In [323]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [324]:
model.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.4019692918285767, details={'was_impossible': False})

In [325]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [326]:
id_map = pd.read_csv('dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [327]:
indices_map = id_map.set_index('id')

In [328]:
def hybrid_recommendations(userId, title):
    title = str(title).title()
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: model.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [330]:
hybrid_recommendations(100, 'toy story')

,title,vote_count,vote_average,year,id,est
2514,Toy Story 2,3914.0,7.3,1999,863,4.096440
7589,Toy Story 3,4710.0,7.6,2010,10193,3.886360
3817,"Monsters, Inc.",6150.0,7.5,2001,585,3.604482
7133,A Matter of Loaf and Death,120.0,7.2,2008,14447,3.566173
8555,The Lego Movie,3127.0,7.5,2014,137106,3.559197
3008,Chicken Run,1190.0,6.5,2000,7443,3.438917
5255,"Bon Voyage, Charlie Brown (and Don't Come Back!)",12.0,6.2,1980,31112,3.232383
8479,Toy Story of Terror!,246.0,7.3,2013,213121,3.225157
840,Pete's Dragon,201.0,6.4,1977,11114,3.221084
6502,Monster House,912.0,6.3,2006,9297,3.214389
